In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [378]:
import pandas as pd
import numpy as np

In [379]:
all_data = pd.read_csv('/content/drive/My Drive/HW3/train.csv')
all_data

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,3,30.40,82.15,Male,0,No,No,No,No phone service,DSL,No,No,No,Yes,No,No,Month-to-month,No,Electronic check,0
5278,50,44.45,2188.45,Male,0,Yes,No,No,No phone service,DSL,Yes,No,No,Yes,Yes,No,One year,Yes,Bank transfer (automatic),0
5279,1,55.05,55.05,Male,0,No,No,Yes,No,DSL,No,No,Yes,Yes,No,No,Month-to-month,No,Mailed check,0
5280,29,76.00,2215.25,Female,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Credit card (automatic),0


**Обработка данных**

Функция `prep(data)` осуществляет подготовку данных, а именно: 

1) заменяет пропущенные значения в столбце $TotalSpent$(в данном случае это пробелы) на нули.

2) если значение ячейки столбца $TotalSpent$ не является пробелом, то проводит изменение типа значения $string\rightarrow float$.

3) выполняет $One-Hot$ $Encoding$.

In [380]:
def prep(data):
  for i in range(len(data.TotalSpent)):
    number = data.TotalSpent[i]
    if(len(number) == 1 and ord(number.encode('windows-1251')) == 32):
      data.TotalSpent[i]=0
    else:
      temp=data.TotalSpent[i]
      data.TotalSpent[i]= float(temp.strip().strip("'"))

  titles = (data.columns.values)
  df1 = data[[titles[0]]]
  df2 = data[[titles[1]]]
  df3 = data[[titles[2]]]
  df1 = df1.join(df2)
  df1 = df1.join(df3)
  for i in range(len(titles)):
    if(i>=3):
      temp = pd.get_dummies(data[[titles[i]]])
      df1 = df1.join(temp)
  return df1

Функция `dec(data, flag)` стандартизирует значения первых трех столбцов.

Если требуется отделить `feature_matrix` и `labels`, то используется `flag`.

In [381]:
def dec(data, flag):
  X = data[['ClientPeriod','MonthlySpending', 'TotalSpent']]
  X = scaler.fit_transform(X)

  if(flag==1):
    labels = data[data.columns[-1]].values
    feature_matrix = data[data.columns[:-1]].values
    feature_matrix = np.hstack((X,feature_matrix[:,3:]))
    return feature_matrix,labels
  elif(flag==0):
    feature_matrix = data.values
    feature_matrix = np.hstack((X,feature_matrix[:,3:]))
    return feature_matrix

In [382]:
all_data = prep(all_data)
all_data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ClientPeriod,MonthlySpending,TotalSpent,Sex_Female,Sex_Male,IsSeniorCitizen,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,HasPhoneService_No,HasPhoneService_Yes,HasMultiplePhoneNumbers_No,HasMultiplePhoneNumbers_No phone service,HasMultiplePhoneNumbers_Yes,HasInternetService_DSL,HasInternetService_Fiber optic,HasInternetService_No,HasOnlineSecurityService_No,HasOnlineSecurityService_No internet service,HasOnlineSecurityService_Yes,HasOnlineBackup_No,HasOnlineBackup_No internet service,HasOnlineBackup_Yes,HasDeviceProtection_No,HasDeviceProtection_No internet service,HasDeviceProtection_Yes,HasTechSupportAccess_No,HasTechSupportAccess_No internet service,HasTechSupportAccess_Yes,HasOnlineTV_No,HasOnlineTV_No internet service,HasOnlineTV_Yes,HasMovieSubscription_No,HasMovieSubscription_No internet service,HasMovieSubscription_Yes,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check,Churn
0,55,19.50,1026.35,0,1,0,0,1,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,1,0,0,0,0,1,0
1,72,25.85,1872.2,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,0,0
2,1,75.90,75.9,0,1,0,1,0,1,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,1
3,32,79.30,2570,1,0,1,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0
4,60,115.25,6758.45,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5277,3,30.40,82.15,0,1,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0
5278,50,44.45,2188.45,0,1,0,0,1,1,0,1,0,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,1,0,0,0,0
5279,1,55.05,55.05,0,1,0,1,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0
5280,29,76.00,2215.25,1,0,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,0


Меняем нули, добавленные вместо пробелов, на среднее значение по столбцу $TotalSpent$.

In [ ]:
for i in range(len(all_data.TotalSpent)):
  if(all_data.TotalSpent[i]==0):
    all_data.TotalSpent[i]=all_data.TotalSpent.mean()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [383]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [384]:
feature_matrix, labels = dec(all_data,1)

In [385]:
from sklearn.linear_model import LogisticRegression

In [396]:
# создание модели с указанием гиперпараметра C
clf = LogisticRegression(C=1,penalty='l1', solver='liblinear')
# обучение модели
clf.fit(feature_matrix, labels)
# предсказание на тестовой выборке

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l1',
                   random_state=None, solver='liblinear', tol=0.0001, verbose=0,
                   warm_start=False)

In [387]:
from sklearn.model_selection import GridSearchCV

Проводим перебор гиперпараметров.

In [388]:
# заново создадим модель, указав солвер
clf = LogisticRegression()

# опишем сетку, по которой будем искать
param_grid = {
    'C': np.arange(1, 5), # также можно указать обычный массив, [1, 2, 3, 4]
    'penalty': ['l1', 'l2'],
    'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']
}

# создадим объект GridSearchCV
search = GridSearchCV(clf, param_grid, n_jobs=-1, cv=5, refit=True, scoring='accuracy')

# запустим поиск
search.fit(feature_matrix, labels)

# выведем наилучшие параметры
print(search.best_params_)

{'C': 1, 'penalty': 'l1', 'solver': 'liblinear'}


In [397]:
test_data = pd.read_csv('/content/drive/My Drive/HW3/test.csv')

In [398]:
test_data = prep(test_data)
test_data

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,ClientPeriod,MonthlySpending,TotalSpent,Sex_Female,Sex_Male,IsSeniorCitizen,HasPartner_No,HasPartner_Yes,HasChild_No,HasChild_Yes,HasPhoneService_No,HasPhoneService_Yes,HasMultiplePhoneNumbers_No,HasMultiplePhoneNumbers_No phone service,HasMultiplePhoneNumbers_Yes,HasInternetService_DSL,HasInternetService_Fiber optic,HasInternetService_No,HasOnlineSecurityService_No,HasOnlineSecurityService_No internet service,HasOnlineSecurityService_Yes,HasOnlineBackup_No,HasOnlineBackup_No internet service,HasOnlineBackup_Yes,HasDeviceProtection_No,HasDeviceProtection_No internet service,HasDeviceProtection_Yes,HasTechSupportAccess_No,HasTechSupportAccess_No internet service,HasTechSupportAccess_Yes,HasOnlineTV_No,HasOnlineTV_No internet service,HasOnlineTV_Yes,HasMovieSubscription_No,HasMovieSubscription_No internet service,HasMovieSubscription_Yes,HasContractPhone_Month-to-month,HasContractPhone_One year,HasContractPhone_Two year,IsBillingPaperless_No,IsBillingPaperless_Yes,PaymentMethod_Bank transfer (automatic),PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,42,56.10,2386.85,0,1,0,0,1,1,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0
1,29,98.50,3004.15,0,1,1,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,1,0,0,1,0
2,9,76.25,684.85,0,1,0,1,0,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0
3,67,74.00,4868.4,1,0,0,0,1,0,1,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0
4,71,20.10,1389.6,1,0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1756,71,105.15,7555,1,0,0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,1,0,0,1,0,1,1,0,0,0
1757,13,20.85,272.35,1,0,0,0,1,0,1,0,1,1,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,0,1
1758,2,91.45,171.45,0,1,0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,1,0
1759,19,89.10,1620.8,1,0,0,0,1,1,0,0,1,0,0,1,0,1,0,1,0,0,0,0,1,1,0,0,1,0,0,1,0,0,0,0,1,1,0,0,0,1,0,0,1,0


In [399]:
test_matrix = dec(test_data,0)

In [400]:
y_pred = clf.predict_proba(test_matrix)

In [401]:
y_pred

array([[0.95902535, 0.04097465],
       [0.28287731, 0.71712269],
       [0.30215855, 0.69784145],
       ...,
       [0.23472788, 0.76527212],
       [0.36542726, 0.63457274],
       [0.98903557, 0.01096443]])

In [402]:
y_sol=y_pred[:,1]

In [404]:
y_sol=y_sol.flatten()

Шаблон для ручной отправки решения:

1) Создаем/ очищаем файл формата .csv

In [ ]:
filename = '/content/drive/My Drive/Kaggle/sub.csv'
# opening the file with w+ mode truncates the file
f = open(filename, "w+")
f.close()

2) Записываем в файл предсказание.

In [ ]:
import csv

with open('/content/drive/My Drive/Kaggle/sub.csv', mode='w') as employee_file:
    employee_writer = csv.writer(employee_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

    employee_writer.writerow([str('Id')+str(',')+str('Churn')])
    for i in range(0, y_pred.shape[0]):
      employee_writer.writerow([str(i)+str(',')+str(y_pred[i,1])])

3) Закрываем файл.

In [ ]:
employee_file.close()

4) Перетаскиваем файл из $Google.drive$ на платформу $Kaggle$.

Шаблон для автоматической отправки решения на $Kaggle$ через $\textbf{API}$:

1) Создаем DataFrame и записываем предсказание.

In [405]:
my_submission = pd.DataFrame({'Id': range(0,1761), 'Churn': y_sol})
# you could use any filename. We choose submission here
my_submission.to_csv('/content/drive/My Drive/Kaggle/submission.csv', index=False)

In [406]:
my_submission

,Id,Churn
0,0,0.040975
1,1,0.717123
2,2,0.697841
3,3,0.005751
4,4,0.001423
...,...,...
1756,1756,0.098814
1757,1757,0.027629
1758,1758,0.765272
1759,1759,0.634573


2) Импортируем токен. 

In [407]:
from google.colab import files
file=files.upload()

Saving kaggle.json to kaggle (1).json


In [408]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


3) Отправляем решение.

In [409]:
!kaggle competitions submit -c advanced-dls-fall-2020 -f submission.csv -m 'commit' -f '/content/drive/My Drive/Kaggle/submission.csv'

100% 41.6k/41.6k [00:06<00:00, 6.23kB/s]
Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python2.7/dist-packages/kaggle/cli.py", line 64, in main
    print(out, end='')
UnicodeEncodeError: 'latin-1' codec can't encode characters in position 26-37: ordinal not in range(256)
